# Segregation calculation

** Instructions**

For fast processing, you can just change the following variables **before running**:
* path/name at **Input file** cell (select the file you want to use)
* bandwidth and weigth method at **compute population intensity** cell
* file name in the variable **fname** at section **Save results to a local file** (the file you want to save results)

*make sure you don't use a name already used or the file will be replaced*

With the previous steps in mind, just click on **Cell** menu and select **Run All**

In [1]:
# Imports
import numpy as np
np.seterr(all='ignore')
import pandas as pd
from decimal import Decimal
import time

# Import python script with Pysegreg functions
from segregationMetrics import Segreg

# Instantiate segreg as cc
cc = Segreg()

# Input file

**Attention to the new data structure for input !!!**

Change your input file with path/name in the cell below to be processed.

**Data Format**  
**ID | X | Y | group 1 | group 2 | group n**

In [2]:
cc.readAttributesFile('/valid/gla_oa_race.csv')

matrix([[  5.47279936e+05,   1.85243551e+05,   1.91000000e+02,
           6.30000000e+01,   1.50000000e+01,   1.90000000e+01],
        [  5.46880923e+05,   1.85295171e+05,   1.95000000e+02,
           6.20000000e+01,   3.60000000e+01,   2.60000000e+01],
        [  5.47089858e+05,   1.85108809e+05,   2.14000000e+02,
           6.90000000e+01,   1.20000000e+01,   3.40000000e+01],
        ..., 
        [  5.26001987e+05,   1.82084880e+05,   1.09000000e+02,
           1.20000000e+01,   4.40000000e+01,   1.20000000e+02],
        [  5.26085638e+05,   1.82223817e+05,   1.38000000e+02,
           8.00000000e+00,   3.20000000e+01,   1.10000000e+02],
        [  5.29567927e+05,   1.79192866e+05,   5.50000000e+01,
           2.20000000e+01,   4.70000000e+01,   8.20000000e+01]])

# Measures

**Compute Population Intensity**

**For non spatial result, please comment the function call at: "cc.locality= ..." **

* to comment a code use # in the begining of the line

Distance matrix is calculated at this step. Change the parameters for the population  
intensity according to your needs. Parameters are:  
* **bandwidth** - is set to be 5000m by default, you can change it here  
* **weightmethod** - 1 for gaussian, 2 for bi-square and empty for moving window

In [3]:
start_time = time.time()
cc.locality = cc.cal_localityMatrix(bandwidth=3000, weightmethod=1)
print("--- %s seconds for processing ---" % (time.time() - start_time))

--- 123.07484102249146 seconds for processing ---


For validation only  
Remove the comment (#) if you want to see the values and validate

In [4]:
# np.set_printoptions(threshold=np.inf)
# print('Location (coordinates from data):\n', cc.location)
# print()
# print('Population intensity for all groups:\n', cc.locality)

'''To select locality for a specific line (validation), use the index in[x,:]'''
# where x is the number of the desired line

# cc.locality[5,:]

'To select locality for a specific line (validation), use the index in[x,:]'

**Compute local Dissimilarity**

In [5]:
diss_local = cc.cal_localDissimilarity()
diss_local = np.asmatrix(diss_local).transpose()

**Compute global Dissimilarity**

In [6]:
diss_global = cc.cal_globalDissimilarity()

** Compute local Exposure/Isolation**  
expo is a matrix of n_group * n_group therefore, exposure (m,n) = rs[m,n]  
the columns are exporsure m1 to n1, to n2... n5, m2 to n1....n5  
- m,m = isolation index of group m
- m,n = expouse index of group m to n

Result of all combinations of local groups expousure/isolation  
To select a specific line of m to n, use the index [x]  
Each value is a result of the combinations m,n  
e.g.: g1xg1, g1xg2, g2,g1, g2xg2 = isolation, expousure, // , isolation

In [7]:
expo_local = cc.cal_localExposure()

** Compute global Exposure/Isolation**

In [8]:
expo_global = cc.cal_globalExposure()

**Compute local Entropy**

In [9]:
entro_local = cc.cal_localEntropy()

**Compute global Entropy**

In [10]:
entro_global = cc.cal_globalEntropy()

**Compute local Index H**

In [11]:
idxh_local = cc.cal_localIndexH()

**Compute global Index H**

In [12]:
idxh_global = cc.cal_globalIndexH()

# Results
**Prepare data for saving on a local file**

In [13]:
# Concatenate local values from measures
if len(cc.locality) == 0:
    results = np.concatenate((expo_local, diss_local, entro_local, idxh_local), axis=1)
else:
    results = np.concatenate((cc.locality, expo_local, diss_local, entro_local, idxh_local), axis=1)

# Concatenate the results with original data
output = np.concatenate((cc.track_id, cc.attributeMatrix, results),axis = 1)

In [14]:
names = ['id','x','y']

for i in range(cc.n_group):
    names.append('group_'+str(i))

if len(cc.locality) == 0:    
    for i in range(cc.n_group):
        for j in range(cc.n_group):
            if i == j:
                names.append('iso_' + str(i) + str(j))
            else:
                names.append('exp_' + str(i) + str(j))
            
    names.append('dissimil')
    names.append('entropy')
    names.append('indexh')
    
else:
    for i in range(cc.n_group):
        names.append('intens_'+str(i))
        
    for i in range(cc.n_group):
        for j in range(cc.n_group):
            if i == j:
                names.append('iso_' + str(i) + str(j))
            else:
                names.append('exp_' + str(i) + str(j))
            
    names.append('dissimil')
    names.append('entropy')
    names.append('indexh')

** Save Local and global results to a file**

The paramenter **fname** corresponds to the folder/filename, change it as you want.  
To save on a diferent folder, use the "/" to pass the directory.  
The local results will be saved using the name defined and adding the **"_local"** postfix to file's name.  
The global results are automatically saved using the same name with the addiction of the postfix **"_globals".**  

It's recommended to save on a different folder from the code, e.g.: a folder named result.

**The fname value should be changed for any new executions or the local file will be overwrited!**

In [15]:
fname = "valid/result_pysegreg"

output = pd.DataFrame(output, columns=names)
output.to_csv("%s_local.csv" % fname, sep=",", index=False)
with open("%s_global.txt" % fname, "w") as f:
    f.write('Global dissimilarity: ' + str(diss_global))
    f.write('\nGlobal entropy: ' + str(entro_global))
    f.write('\nGlobal Index H: \n' + str(idxh_global))
    f.write('\nGlobal isolation/exposure: \n')
    f.write(str(expo_global))